# A word for the beginning

this was an experiment - did not expect to find a good solution, did not even bother to improve the data and make proper outputs. For each 'dimention' of sudoku there's a conv layer, one going in squares, one in horizontal lines, one in vertical, then concatenated and dense layer in the end. Unfortunately whatever architecture did I use, there is no shortcut for sudoku. The only solution I found to work is a recurrent network, calculating probability for each empty square for each number, then a filling one square that had a number with highest probability from the entire grid, and then calculating probability again this time with the square filled.

Sudoku is an NP problem, and the only way to improve the process and fill the grid and any larger variation (16x16, 25x25...) with the same P time would be to calculate every single cell for any possible grid combinations, at the same time, which is not possible on our boring deterministic machines with finite memory and speed. 

Training data: 
https://www.kaggle.com/bryanpark/sudoku 1 000 000 sudokus, unsolved + solved

In [ ]:
import pandas as pd
import numpy as np

import keras
from keras import models
from keras.layers import *
from keras.utils import to_categorical
from tensorflow import nn

import matplotlib.pyplot as plt
from random import randint

from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
#Ścieżka do pliku
data = pd.read_csv('gdrive/My Drive/Colab Notebooks/sudoku/sudoku.csv', sep = ',')

Mounted at /content/gdrive


In [ ]:
listify = lambda x : np.array([(int(i) - 5)/5 for i in x], dtype=float)
npdata = data.applymap(listify)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(np.array(npdata.iloc[:,0]), np.array(npdata.iloc[:,1]), test_size = 0.2)

In [ ]:
x_train = np.reshape(np.array(x_train.tolist()), (800000,9,9))
x_test = np.reshape(np.array(x_test.tolist()), (200000,9,9))
y_train = np.reshape(np.array(y_train.tolist()), (800000,9,9))
y_test = np.reshape(np.array(y_test.tolist()), (200000,9,9))

In [ ]:
y_train[0]

array([[ 0. ,  0.2, -0.2,  0.8, -0.8,  0.6,  0.4, -0.4, -0.6],
       [-0.4,  0.6, -0.6, -0.2,  0.4,  0.2,  0.8, -0.8,  0. ],
       [ 0.4, -0.8,  0.8, -0.4, -0.6,  0. ,  0.6,  0.2, -0.2],
       [-0.8, -0.2, -0.4,  0. ,  0.6,  0.4, -0.6,  0.8,  0.2],
       [-0.6,  0.8,  0.6,  0.2, -0.4, -0.2, -0.8,  0. ,  0.4],
       [ 0.2,  0. ,  0.4, -0.8,  0.8, -0.6, -0.4, -0.2,  0.6],
       [-0.2, -0.6, -0.8,  0.6,  0. , -0.4,  0.2,  0.4,  0.8],
       [ 0.8,  0.4,  0.2, -0.6, -0.2, -0.8,  0. ,  0.6, -0.4],
       [ 0.6, -0.4,  0. ,  0.4,  0.2,  0.8, -0.2, -0.6, -0.8]])

In [ ]:
inputlayer = Input((9,9,1))
squares = Conv2D(9, (3,3), strides = (3,3), input_shape = (9,9,1), activation = 'softmax', padding = 'valid')(inputlayer)
flat_squares = Flatten()(squares)
squares_dense = Dense(9, activation = 'softmax')(flat_squares)

cols = Conv2D(9, (9,1), strides = (1,1), input_shape = (9,9,1), activation='softmax', padding = 'valid')(inputlayer)
flat_cols = Flatten()(cols)
cols_dense = Dense(9, activation = 'softmax')(flat_cols)

rows = Conv2D(9, (1,9), strides = (1,1), input_shape = (9,9,1), activation = 'softmax', padding = 'valid')(inputlayer)
flat_rows = Flatten()(rows)
rows_dense = Dense(9, activation = 'softmax')(flat_rows)

merge = concatenate([squares_dense, cols_dense, rows_dense], axis = 1)

dense = Dense(81, activation = 'softmax')(merge)
out = Reshape((9,9,1))(dense)

model = keras.Model(inputlayer, out)

model.compile(optimizer = keras.optimizers.Adam(), loss='categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs = 10)

Epoch 1/10
25000/25000 [==============================] - 65s 3ms/step - loss: -3.1188 - accuracy: 0.1109
Epoch 2/10
25000/25000 [==============================] - 72s 3ms/step - loss: -5.3556 - accuracy: 0.1108
Epoch 3/10
25000/25000 [==============================] - 72s 3ms/step - loss: -5.9862 - accuracy: 0.1108
Epoch 4/10
25000/25000 [==============================] - 71s 3ms/step - loss: -6.5904 - accuracy: 0.1108
Epoch 5/10
25000/25000 [==============================] - 68s 3ms/step - loss: -7.0535 - accuracy: 0.1108
Epoch 6/10
25000/25000 [==============================] - 67s 3ms/step - loss: -7.3725 - accuracy: 0.1107
Epoch 7/10
25000/25000 [==============================] - 69s 3ms/step - loss: -7.6257 - accuracy: 0.1107
Epoch 8/10
25000/25000 [==============================] - 67s 3ms/step - loss: -7.8118 - accuracy: 0.1107
Epoch 9/10
25000/25000 [==============================] - 65s 3ms/step - loss: -7.9495 - accuracy: 0.1107
Epoch 10/10
25000/25000 [=====================